In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import datetime
from scipy.optimize import minimize
import time
from math import comb

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.

#### Strategy
We chose the safe option as our strategy. Our strategy makes use of two main principles in portfolio management. The first is that given stocks with low correlation, their aggregate has an overall lower standard deviation to its mean. This is volatility inherent to each stock, better known as non-systematic risk, “cancels out” each other. The second principle is that given a set of stock weightings, there exists a set of weights (within minimum/maximum bounds) that creates an expected return that exactly matches the market’s return. Given the weights to achieve that expected return, a tracking error can be generated. 
Generating low-correlated portfolios is done as follows: we know that low correlation comes in pairs, so given a dataframe we pick pairs of tickers with the lowest correlations, removing those tickers from the correlation matrix when they are selected. This generates a list of 12 pairs of low-correlated stocks. We then pick 5 random combinations for each of 6, 7, 8, 9, 10, and 11 pairs, and one combination for 12 pairs. This gives us a healthy pool of portfolios of different sizes to work with, while not being too computationally intensive because it results in a maximum of 31 (5 samples * 6 portfolio sizes + 1) portfolios to test from.
We’re looking to compare a variety of low-correlation portfolios optimized to minimize tracking error compared to the market. Then, because those portfolio weights changed, the expected mean return of the portfolio also changed. Now, we compare the difference between the portfolio’s expected mean and the market mean and look for the smallest discrepancy. That gives us a low-correlation portfolio (indicating generally low portfolio standard deviation) optimized for the lowest tracking error, as well as the closest average return to the market mean. With these factors in mind, we meet the market the best and return that portfolio.


#### Function Dictionary
validity(tickers) takes a list of tickers and filters for all the valid tickers in the list.
<HELPERS>
> check_delist(ticker) returns true if the ticker is delisted and false otherwise.

> check_currency(ticker) returns true if the currency of the ticker is either in CAD or USD.

> check_volume(ticker) returns true if the stock has an average monthly volume of 100 000 shares, false otherwise.

> check_first_day(ticker) returns true if the stock has a listed value on the first day. False otherwise.

> check_last_day(ticker) returns true if the stock has a listed value on the last day. False otherwise.

> check_target_day(ticker) returns true if the stock has a listed value on the target day. False otherwise.

portfolio_generator(tickers) takes a portfolio representing a list of tickers, 
fetches the close price of the stocks on November 22 to be the buy price for each stock on the ticker,
and given a portfolio made up of those tickers, the function optimizes portfolio weights so that the mean
expected return of the market is met. All fees are taken into account. It will then spit out all the
tickers created with their respective weights, and chooses the portfolio with the lowest standard deviation 
to be our final choice of tickers.

days_to_drop(markets) takes a dataframe of stock returns, and drops values in any month that doesn't have 18 or more trading days.

std_rank(returns) takes a dataframe of stock returns, and returns the 24 tickers of that stock return dataframe with the lowest standard deviation of percent returns.

yfin_pull_convert_USD(ticker,start,end) takes a ticker string and returns the stock returns of that ticker between the start and end date, converted to CAD if listed in USD.

gen_tickers(tlist,ilist) takes a list of tickers and a list of index tickers, generating the exchange-rate-adjusted stock values over a time period of all the tickers.

convert_pct_returns(stock_value_df) takes a dataframe of stock values and converts them into percent returns, interpolating anything in between.

correl(data, dropvalue) takes a dataframe of percent returns, drops the columns specified in dropvalue (going to be the index tickers) and creates a correlation matrix based on the remaining columns.

low_correl_ticker_pairs(correlation matrix) takes a correlation matrix and chooses the lowest correlation pair of tickers.

corr_pair_extract(correl_matrix) takes a correlation matrix and continuously runs low_correl_ticker_pairs, removing the ticker pairs from the matrix along the way. 

make_port_list(stock_pairs) takes a list of stock pairs and randomly chooses 5 samples each of 6-12 (or however many pairs are available) as final portfolios.

portfolio_generator() optimizes each of the portfolios generated from make_port_list for lowest tracking error. This is so we can then compare the mean returns and find the portfolio with the lowest difference between its mean return and the market mean return. USED AI.

In [2]:
#October 01, 2023 to September 30, 2024, 2023

START_DATE = pd.to_datetime('2023-10-01')
END_DATE = pd.to_datetime('2024-10-01')

# date on which we will take stock Close prices
TARGET_DATE = pd.to_datetime('2024-11-22')

INVESTMENT = 1e6

CSV_FILE = "Tickers.csv"

#Arbitrarily assign a seed to keep results consistent
random.seed(123)

In [3]:
# read in CSV
csv = pd.read_csv(CSV_FILE, header=None)

In [4]:
# filter out invalid stocks
def validity(tickers):

    # list to keep all of the valid stocks
    final_list = []
    stock_names = []

    for ticker in tickers:
        #only append the stock to the final stock list if it is isn't delisted, has a US or CAD currency, and has the required monthly share volume
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_first_day(ticker) and
            check_last_day(ticker) and
            check_target_day(ticker) and
            check_volume(ticker) and
            # NEW CODE
            check_is_stock(ticker) and
            ticker not in final_list):
                # NEW CODE
                loc_in_final_list = check_dupes(ticker, stock_names)
                # NEW CODE
                if loc_in_final_list != -1:
                    if should_switch_stock(final_list[loc_in_final_list], ticker):
                        # NEW CODE
                        final_list.remove(final_list[loc_in_final_list])
                        stock_names.remove(stock_names[loc_in_final_list])
                        # NEW CODE
                        final_list.append(ticker)
                        stock_names.append(yf.Ticker(str(ticker)).info['longName'])
                else:
                    final_list.append(ticker)
                    stock_names.append(yf.Ticker(str(ticker)).info['longName'])
    return final_list

# checks if the stock exists on the last day of the period
def check_target_day(ticker):
    stock = yf.Ticker(str(ticker))
    data = stock.history(start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1))
    # time.sleep(0.3)
    if data.empty:
        return False
    return True

# checks if the stock exists on the target day
def check_last_day(ticker):
    stock = yf.Ticker(str(ticker))
    data = stock.history(start=END_DATE-datetime.timedelta(days=5), end=END_DATE)
    # time.sleep(0.3)
    if data.empty:
        return False
    return True

# checks if the stock exists on the first day of the period
def check_first_day(ticker):
    stock = yf.Ticker(str(ticker))
    data = stock.history(start=START_DATE, end=START_DATE+datetime.timedelta(days=5))
    # time.sleep(0.3)
    if data.empty:
        return False
    return True

# checks if the consumed Ticker is delisted
def check_delist(ticker):
    stock = yf.Ticker(str(ticker))
    try:
        data = stock.history(period='1d')
        # time.sleep(0.3)
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock
            if 'Close' not in data.columns or data['Close'].isnull().all():
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

# checks if the consumed Ticker meets the requirement of an average monthly share volume of 100,000 shares
def check_volume(ticker):
    volume = yf.Ticker(str(ticker)).history(start=START_DATE, end=END_DATE)['Volume']
    # NEW CODE
    volume.index = pd.to_datetime(pd.to_datetime(volume.index).strftime('%Y-%m-%d'))
    time.sleep(0.3)
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

# checks whether or not the consumed Ticker is listed in CAD or in USD
def check_currency(ticker):
    stock = yf.Ticker(str(ticker))
    currency = stock.fast_info.get('currency')
    return currency in ['USD', 'CAD']

# NEW CODE
# checks if the ticker is already in the final list
def check_dupes(ticker, stock_names):
    name = yf.Ticker(str(ticker)).info["longName"]
    if name in stock_names:
        return stock_names.index(name)
    return -1

# NEW CODE
# Checks if the ticker should be swapped depending on the prices
def should_switch_stock(current, new):
    new = yf.Ticker(str(new)).history(start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1))['Close']
    current = yf.Ticker(str(current))
    current_hist = current.history(start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1))['Close']
    exch_rate = yf.Ticker('CADUSD=x').history(start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1))['Close']

    if current.info["currency"] == 'USD':
        return current_hist.iloc[0] / exch_rate.iloc[0] < new.iloc[0]
    return current_hist.iloc[0] * exch_rate.iloc[0] < new.iloc[0]

# NEW CODE
# Checks if the ticker is a stock
def check_is_stock(ticker):
    if yf.Ticker(str(ticker)).info['quoteType'] != "EQUITY":
        return False
    return True
    
# create a new DataFrame only containing the valid Tickers in the consumed CSV
tickers_raw = validity(csv[0])

$ASDFAASDF.TO: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$INVALIDTIC: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$HDFC.NS: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AW.TO: possibly delisted; no price data found  (1d 2023-10-01 00:00:00 -> 2023-10-06 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1696132800, endDate = 1696564800")


<span style="color: red;">

Add function check_dupes that checks if a stock already exists in the list of stocks. Will return the index of the stock if it does, -1 if it does not. Will return the index if the ticker of a stock on another market is present.

Add should_switch_stock, which compares the prices of a stock in two markets. Returns true if the already converted price of the existing ticker in the list is less than the converted price of the new ticker. False otherwise.

Add check_is_stock function that checks if a ticker is actually a stock. Returns true if it is and false if it not.

Add conditions to the validity function to ensure all bases are covered

</span>

In [5]:
# finds all the days that are only on one of the markets
def days_to_drop(markets):
    markets.dropna(inplace=True)    # drop any days where either market has no listed value
    months = markets.resample('MS').bfill().index    # Get the first day of each month in the timeframe
    
    dates = []
    for month in months:
        # Find all listed days within a month 
        temp = markets[(markets.index.month==month.month) & (markets.index.year==month.year)].index
        if len(temp) < 18:    # If the month has less than 18 trading days, add all the days in the month to dates
            dates.extend(temp)
    
    return dates

In [6]:
tickers_raw_df = pd.DataFrame()

# calculate and store all returns for all all Tickers
for ticker in tickers_raw:
    data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close'].pct_change()
    time.sleep(0.3)
    data.drop(index=data.index[0], inplace=True)
    tickers_raw_df[ticker] = data  # this code calculates daily returns

tickers_raw_df.index = pd.to_datetime(tickers_raw_df.index.strftime("%Y-%m-%d"))

# tickers_raw_df

In [7]:
def std_rank(returns):
    std_devs = returns.std()
    
    # Create a DataFrame with tickers and their respective standard deviations
    ranked_stocks = pd.DataFrame({
        "ticker": std_devs.index,
        "STD": std_devs.values
    })
    
    # Sort by standard deviation in ascending order (lowest STD at the top)
    ranked_stocks = ranked_stocks.sort_values(by="STD", ascending=True)
    
    return ranked_stocks['ticker'].values[:24].tolist()

ticker_lst = std_rank(tickers_raw_df)

In [8]:
index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start=TARGET_DATE, period='1d')["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker, start, end):
    listed_currency = ticker.fast_info.get('currency')
    if listed_currency == "CAD":
        temp = ticker.history(start=start, end=end, interval="1d")["Close"]
        temp.index = pd.to_datetime(temp.index.strftime('%Y-%m-%d'))
        return temp
    elif listed_currency == "USD":
        temp = ticker.history(start=start, end=end, interval="1d")["Close"] * (1/exch_rate)
        temp.index = pd.to_datetime(temp.index.strftime('%Y-%m-%d'))
        return temp
    raise ValueError("Currency not in CAD or USD")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    check_month_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        check_month_dataframe[i_str] = yfin_pull_convert_USD(index_ticker, start=START_DATE, end=END_DATE)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker, start=START_DATE, end=END_DATE)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        time.sleep(0.3)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker, start=START_DATE, end=END_DATE)
    ret_dataframe.drop(days_to_drop(check_month_dataframe), inplace=True)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst).resample('W-MON').bfill()
stock_values.index = pd.to_datetime(stock_values.index.strftime('%Y-%m-%d'))
# display(stock_values)

[*********************100%***********************]  1 of 1 completed


In [9]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)
# display(stock_returns)

In [10]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
# display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


In [11]:
#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)

In [12]:
#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    for i in range(6, len(stock_pairs)+1):
        #simple iteration, of 5 times
        for j in range(min(comb(len(stock_pairs), i), 5)):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

portfolio_list = make_port_list(stock_pairs)

In [14]:
# Helps if the weights add up to one
def sum_weights(weights):
    return sum(weights) - 1

# Generates multiple portfolios out of 2d list of tickers given
def portfolio_generator():
    portfolio_data = []

    for portfolio in portfolio_list:
        tickers = portfolio

        # Evenly weigh each stock for now
        initial_weights = [1 / len(tickers)] * len(tickers)

        constraints = [{'type':'eq',
                        'fun': sum_weights}]

        # Sets upper and lower boundaries on the weighting
        bounds = [(1 / (2 * len(tickers)), 0.15)] * len(tickers)

        # Set the objective to minimize tracking error of the portfolio
        def objective(weights):
            return np.std(np.dot(stock_returns[tickers].values, weights) - stock_returns['Market Returns'].values)

        # Mimimize tracking error
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)

        if result.success:
            weights = result.x

            # Calculate the amount of money spent on each stock
            portfolio_value = INVESTMENT * weights
            
            # Calculate the amount of each share to purchase and store in an array
            shares = portfolio_value / [yfin_pull_convert_USD(yf.Ticker(ticker), start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1)).values[0] for ticker in tickers]
            
            # Calculate the fees for each stock and store them in an array
            fees = np.minimum(3.95, 0.001 * shares)

            portfolio_value -= fees

            # Recalculate shares to purchase after subtracting the fees from the amount to invest into each stock
            shares = portfolio_value / [yfin_pull_convert_USD(yf.Ticker(ticker), start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1)).values[0] for ticker in tickers]

            portfolio_data.append({
                'tickers': tickers,  # Tickers in the portfolio
                'weights': weights,  # Optimized weights
                'Tracking Error': result.fun,  # Standard deviation (risk)
                'Difference of Mean Returns': abs(np.dot(stock_returns[tickers].values, weights).mean() - stock_returns['Market Returns'].values.mean()),
                'num_stocks': len(portfolio),  # Number of stocks in the portfolio 
                'total_cost': portfolio_value.sum() + sum(fees),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_value.sum(),  # Total portfolio value after fees
                'final_fees': sum(fees)
            })

    return pd.DataFrame(portfolio_data)
            

portfolio_df = portfolio_generator()
# portfolio_df['weights'].iloc[0]

# display(portfolio_df)

In [18]:
portfolio_df

,tickers,weights,Tracking Error,Difference of Mean Returns,num_stocks,total_cost,shares_purchased,portfolio_value,final_fees
0,"[T.TO, AMZN, SO, OXY, ABBV, AAPL, IBM, GM, BK,...","[0.13871281371574704, 0.13027348554538723, 0.0...",0.006339,0.000424,12,1000000.0,"[6524.405560791585, 472.73375395301645, 447.65...",999989.974621,10.025379
1,"[T.TO, AMZN, IBM, GM, JPM, CSCO, TD.TO, SPG, L...","[0.13462341861896548, 0.14432462709216595, 0.0...",0.007342,0.000511,12,1000000.0,"[6332.053959062716, 523.7222483724713, 133.669...",999990.386669,9.613331
2,"[T.TO, AMZN, COST, CVS, BK, CMCSA, LOW, VZ, JP...","[0.15, 0.13747803792165816, 0.1374907670770005...",0.007309,0.000059,12,1000000.0,"[7055.3174216917205, 498.87748593444945, 102.0...",999990.414236,9.585764
3,"[PEP, GOOG, COST, CVS, T.TO, AMZN, IBM, GM, AB...","[0.04166666666666667, 0.11311608843485446, 0.0...",0.006118,0.000096,12,1000000.0,"[183.97747811728544, 485.75641449527757, 30.91...",999990.858706,9.141294
4,"[IBM, GM, ABBV, AAPL, T.TO, AMZN, LOW, VZ, RY....","[0.041666666666666664, 0.042523631465687675, 0...",0.005863,0.000053,12,1000000.0,"[133.66994338908754, 519.6840146925172, 291.59...",999989.506205,10.493795
5,"[T.TO, AMZN, ABBV, AAPL, COST, CVS, TD.TO, SPG...","[0.14182798729900234, 0.13605738046110075, 0.0...",0.005221,0.000436,14,1000000.0,"[6670.933009037955, 493.7222332627407, 268.644...",999991.206852,8.793148
6,"[TD.TO, SPG, RY.TO, SLB, SO, OXY, BK, CMCSA, T...","[0.08378618235953737, 0.03571428571428571, 0.0...",0.007298,0.000242,14,1000000.0,"[1067.1903308603844, 141.03234496255953, 523.9...",999988.812451,11.187549
7,"[LOW, VZ, JPM, CSCO, RY.TO, SLB, PEP, GOOG, AX...","[0.12427671256450468, 0.03571428571428572, 0.1...",0.005793,0.000113,14,1000000.0,"[335.861562597749, 592.0341915503776, 314.0330...",999990.706693,9.293307
8,"[BK, CMCSA, JPM, CSCO, IBM, GM, AXP, BAC, RY.T...","[0.03571428571428571, 0.03571428571428573, 0.0...",0.009274,0.000094,14,1000000.0,"[318.77304634504753, 587.6760678374795, 277.92...",999991.433177,8.566823
9,"[IBM, GM, LOW, VZ, JPM, CSCO, T.TO, AMZN, COST...","[0.03571428571428571, 0.03571428571428572, 0.0...",0.007274,0.000527,14,1000000.0,"[114.57423719064647, 436.46656558134737, 145.1...",999990.122433,9.877567


In [67]:
temp = portfolio_df[portfolio_df['Tracking Error'] == portfolio_df['Tracking Error'].min()]


# Define columns for the final portfolio DataFrame
portfolio = []

# Adds the individual stock data to the final dataframe
for i in range(len(temp['tickers'].iloc[0])):
    price = yfin_pull_convert_USD(yf.Ticker(temp['tickers'].iloc[0][i]), start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1)).values[0]
    portfolio.append({
        "Ticker": temp['tickers'].iloc[0][i],
        "Price": price,
        "Currency": yf.Ticker(temp['tickers'].iloc[0][i]).info['currency'],
        "Shares": temp['shares_purchased'].iloc[0][i],
        "Value": price * temp['shares_purchased'].iloc[0][i],
        "Weight": temp['weights'].iloc[0][i]})

Portfolio_Final = pd.DataFrame(portfolio)

# Reset index to start at 1
Portfolio_Final.index = Portfolio_Final.index + 1

# Find total weight and total value
total = (Portfolio_Final['Price'] * Portfolio_Final['Shares']).sum() + temp['final_fees'].values[0]
sum_weight = sum(Portfolio_Final['Weight']) * 100
print(f"Total value (with fees added): {total}")
print(f"Sum of Weights: {sum_weight}%")


display(Portfolio_Final)

Total value (with fees added): 999999.9999999998
Sum of Weights: 100.0%


,Ticker,Price,Currency,Shares,Value,Weight
1,T.TO,21.260000,CAD,5677.339266,120700.234100,0.120704
2,AMZN,275.573749,USD,418.180351,115239.526984,0.115240
3,JPM,347.472899,USD,159.564697,55444.407680,0.055445
4,CSCO,81.852898,USD,339.358034,27777.438416,0.027778
5,LOW,370.022624,USD,195.579772,72368.940267,0.072369
6,VZ,60.323701,USD,460.471038,27777.317299,0.027778
7,IBM,311.712057,USD,95.508387,29771.115709,0.029771
8,GM,81.824937,USD,339.473995,27777.438300,0.027778
9,TD.TO,78.510002,CAD,974.152172,76480.689104,0.076482
10,SPG,253.233715,USD,109.691824,27777.668086,0.027778


In [68]:
close_prices = []
for index in Portfolio_Final.index:
    if Portfolio_Final['Currency'].iloc[index-1] == 'CAD':
        close_prices.append(Portfolio_Final['Shares'].iloc[index-1] * yf.Ticker(Portfolio_Final['Ticker'].iloc[index-1]).history(start=TARGET_DATE+datetime.timedelta(days=7), end=TARGET_DATE+datetime.timedelta(days=8)).Close.values[0])
    else:
        close_prices.append(Portfolio_Final['Shares'].iloc[index-1] * yf.Ticker(Portfolio_Final['Ticker'].iloc[index-1]).history(start=TARGET_DATE+datetime.timedelta(days=7), end=TARGET_DATE+datetime.timedelta(days=8)).Close.values[0] / yf.Ticker('CADUSD=x').history(start=TARGET_DATE+datetime.timedelta(days=7), end=TARGET_DATE+datetime.timedelta(days=8)).Close.values[0])

np.sum(close_prices)


1023409.7370615206

In [69]:
tsx1 = yf.Ticker('XIU.TO').history(start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1)).Close.values[0]
tsx2 = yf.Ticker('XIU.TO').history(start=TARGET_DATE+datetime.timedelta(days=7), end=TARGET_DATE+datetime.timedelta(days=8)).Close.values[0]
nasdaq1 = yf.Ticker('^GSPC').history(start=TARGET_DATE, end=TARGET_DATE+datetime.timedelta(days=1)).Close.values[0]
nasdaq2 = yf.Ticker('^GSPC').history(start=TARGET_DATE+datetime.timedelta(days=7), end=TARGET_DATE+datetime.timedelta(days=8)).Close.values[0]
avg1 = (tsx1+nasdaq1)/2
avg2 = (tsx2+nasdaq2)/2
ret = (avg2-avg1)/avg1

ret

0.010554706777939864

In [ ]:
# Creates a dataframe containing only the stock tickers and the number of shares and outputs it to a csv file
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv("Stocks_Group_12.csv", sep='\t', encoding='UTF-8')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Alex, Derek, & Sharuga